In [1]:
img = '/mnt/gsdata/projects/panops/Labeled_data_seprated_in_Folder/image_all'
img_mask = '/mnt/gsdata/projects/panops/Labeled_data_seprated_in_Folder/image_mask_all'

In [3]:
from torch.utils.data import DataLoader
from transformers import MaskFormerImageProcessor

def get_preprocessor():
    # Create a preprocessor
    return MaskFormerImageProcessor(ignore_index=0, reduce_labels=False, do_resize=False, do_rescale=False, do_normalize=False)


/home/as2114/miniconda3/envs/mask2former/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def collate_fn(batch, preprocessor=get_preprocessor()):
    inputs = list(zip(*batch))
    images = inputs[0]
    segmentation_maps = inputs[1]
    # this function pads the inputs to the same size,
    # and creates a pixel mask
    # actually padding isn't required here since we are cropping
    batch = preprocessor(
        images,
        segmentation_maps=segmentation_maps,
        return_tensors="pt",
    )

    batch["original_images"] = inputs[2]
    batch["original_segmentation_maps"] = inputs[3]
    
    return batch


/tmp/ipykernel_717829/4167767847.py:6: FutureWarning: `reduce_labels` is deprecated and will be removed in version 4.44.0 for `MaskFormerImageProcessor.__init__`. Use `do_reduce_labels` instead.
  return MaskFormerImageProcessor(ignore_index=0, reduce_labels=False, do_resize=False, do_rescale=False, do_normalize=False)


In [5]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        #for mask name append mask_ in front of image name
        mask_name = 'mask_'+img_name
        
        print(f'idx: {idx} img name :{img_name} mask_name :{mask_name}')
        
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, mask_name)
        
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Assuming mask is in grayscale
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        return image, mask



idx: 41385 img name :Fagus sylvatica0003446.jpg mask_name :mask_Fagus sylvatica0003446.jpgidx: 67290 img name :Quercus petraea0001003.jpg mask_name :mask_Quercus petraea0001003.jpgidx: 75067 img name :Sorbus aucuparia0007000.jpg mask_name :mask_Sorbus aucuparia0007000.jpg


idx: 45194 img name :Fagus sylvatica0007122.jpg mask_name :mask_Fagus sylvatica0007122.jpg
idx: 17426 img name :Aesculus hippocastanum0007424.jpg mask_name :mask_Aesculus hippocastanum0007424.jpg
idx: 63847 img name :Prunus avium0007600.jpg mask_name :mask_Prunus avium0007600.jpg
idx: 70450 img name :Sorbus aucuparia0002554.jpg mask_name :mask_Sorbus aucuparia0002554.jpg

idx: 18428 img name :Aesculus hippocastanum0008426.jpg mask_name :mask_Aesculus hippocastanum0008426.jpg


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/as2114/miniconda3/envs/mask2former/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/as2114/miniconda3/envs/mask2former/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/as2114/miniconda3/envs/mask2former/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_744704/1394133895.py", line 27, in __getitem__
    mask = Image.open(mask_path).convert("L")  # Assuming mask is in grayscale
  File "/home/as2114/miniconda3/envs/mask2former/lib/python3.8/site-packages/PIL/Image.py", line 3277, in open
    fp = builtins.open(filename, "rb")
FileNotFoundErr

idx: 66204 img name :Prunus avium0009957.jpg mask_name :mask_Prunus avium0009957.jpg

In [ ]:
def build_loader(config):
    
    train_dataset = config.train
    train_config = config.meta[train_dataset]

    train_dat, val_dat, test_dat, columns = parse_dataset(train_config)


    dataset_train = build_dataset(is_train=True, dataframe=train_dat, config=config, col=columns, modality=modality)
    print('successfully build train dataset')

    dataset_test = build_dataset(is_train=False, dataframe=test_dat, config=config, col=columns, modality=modality)
    print(f'local rank {local_rank} / global rank {dist.get_rank()} \
        successfully build test dataset')
    
    
    
    sampler_test = torch.utils.data.SequentialSampler(dataset_test)
    
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        sampler=sampler_train,
        batch_size=config.batch_size,
        num_workers=8,
        pin_memory=True,
        persistent_workers=True,
        collate_fn=collate_fn ### NOTEL THIS ###
    )

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test,
        sampler=sampler_test,
        batch_size=config.batch_size,
        num_workers=8,
        pin_memory=True,
        persistent_workers=True,
        collate_fn=collate_fn ### NOTEL THIS ###
    )


    return dataset_train, data_loader_train
